In [1]:
import importlib
import torch
import utility
from utility import LeNet
from autoattack import AutoAttack

Re-run the following code to re-import the utility importlib.reload(utility)

In [2]:
net = utility.network()
net.cuda()

LeNet(
  (cnn1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cnn2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [3]:
orignal_samples = utility.samples(adversarial=False)
utility.evaluate(net, orignal_samples, defense=utility.trimmer, device=torch.device("cuda"))

Check accuracy of the model: 1.0


In [4]:
import torch.nn as nn

class LeNet_BPDA(torch.nn.Module):
    def __init__(self, pretrained_model, defense):
        super(LeNet_BPDA, self).__init__()
        self.pretrained_model = pretrained_model
        self.defense = defense
    
    def forward(self, x):
        defense_val = self.defense(x, edge=0.3)
        x_rec = x.clone()
        x_rec.data = defense_val.data
        out = self.pretrained_model(x_rec)
        return out

net_bpda = LeNet_BPDA(net, utility.trimmer)
net_bpda.cuda()

LeNet_BPDA(
  (pretrained_model): LeNet(
    (cnn1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (cnn2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=3136, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

Step 1: set-up BPDA attacks 

In [5]:
inf_BPDA = AutoAttack(net_bpda, norm='Linf', eps=0.3, version='standard')
# inf_BPDA.attacks_to_run = ['apgd-ce']
inf_BPDA.apgd.n_restarts = 5
inf_BPDA.apgd.n_iter = 100

setting parameters for standard version


Step 2: apply the APGD

In [6]:
for images, labels in orignal_samples:
    adv_complete = inf_BPDA.run_standard_evaluation(images, labels, bs=100)

using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 100.00%
apgd-ce - 1/1 - 71 out of 100 successfully perturbed
robust accuracy after APGD-CE: 29.00% (total time 1.7 s)
apgd-t - 1/1 - 3 out of 29 successfully perturbed
robust accuracy after APGD-T: 26.00% (total time 5.8 s)
fab-t - 1/1 - 0 out of 26 successfully perturbed
robust accuracy after FAB-T: 26.00% (total time 13.6 s)
square - 1/1 - 1 out of 26 successfully perturbed
robust accuracy after SQUARE: 25.00% (total time 22.1 s)
max Linf perturbation: 0.30000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 25.00%


In [7]:
BPDA_sample = utility.samples(x=adv_complete)

In [8]:
utility.evaluate(net, BPDA_sample, device=torch.device("cuda"))

Check accuracy of the model: 0.25


In [9]:
utility.evaluate(net_bpda, BPDA_sample, device=torch.device("cuda"))

Check accuracy of the model: 0.25


Load the adversarial trained model

In [10]:
net_adv = torch.load("./data/LeNet_32_adv.pt")
net_adv.eval()
net_adv.cuda()

LeNet(
  (cnn1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (cnn2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3136, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

In [11]:
utility.evaluate(net_adv, BPDA_sample, device=torch.device("cuda"))

Check accuracy of the model: 0.97


In [12]:
class LeNet_BPDA_Skip(torch.nn.Module):
    def __init__(self, pretrained_model):
        super(LeNet_BPDA_Skip, self).__init__()
        self.pretrained_model = pretrained_model
    
    def forward(self, x):
        x = self.pretrained_model.cnn1[0](x)
        skip_val = self.pretrained_model.cnn1[1](x)
        x_rec = x.clone()
        x_rec.data = skip_val.data
        out = self.pretrained_model.cnn1[2](x_rec)
        
        out = self.pretrained_model.cnn2(out)
        out = out.view(out.size(0),-1)
        
        out = self.pretrained_model.fc1(out)
        out = self.pretrained_model.fc2(torch.relu(out))
        return out

net_bpda_skip = LeNet_BPDA_Skip(net_adv)
net_bpda_skip.cuda()

LeNet_BPDA_Skip(
  (pretrained_model): LeNet(
    (cnn1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (cnn2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (fc1): Linear(in_features=3136, out_features=1024, bias=True)
    (fc2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

In [13]:
inf_BPDA_Skip = AutoAttack(net_bpda_skip, norm='Linf', eps=0.3, version='standard')
# inf_BPDA_Skip.attacks_to_run = ['apgd-ce']
inf_BPDA_Skip.apgd.n_restarts = 1
inf_BPDA_Skip.apgd.n_iter = 100

setting parameters for standard version


In [14]:
for images, labels in orignal_samples:
    adv_complete_Skip = inf_BPDA_Skip.run_standard_evaluation(images, labels, bs=100)

using standard version including apgd-ce, apgd-t, fab-t, square
initial accuracy: 100.00%
apgd-ce - 1/1 - 3 out of 100 successfully perturbed
robust accuracy after APGD-CE: 97.00% (total time 0.3 s)
apgd-t - 1/1 - 2 out of 97 successfully perturbed
robust accuracy after APGD-T: 95.00% (total time 4.5 s)
fab-t - 1/1 - 0 out of 95 successfully perturbed
robust accuracy after FAB-T: 95.00% (total time 13.7 s)
square - 1/1 - 2 out of 95 successfully perturbed
robust accuracy after SQUARE: 93.00% (total time 23.9 s)
max Linf perturbation: 0.30000, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 93.00%
